In [7]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv('Legally_Operating_Businesses.csv', keep_default_na=True, delimiter=',', skipinitialspace=True)

C:\Users\simon\AppData\Local\Temp\ipykernel_29160\2642446586.py:1: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Legally_Operating_Businesses.csv', keep_default_na=True, delimiter=',', skipinitialspace=True)


In [9]:
# Convert 'License Expiration Date' and 'License Creation Date' to datetime
df['License Expiration Date'] = pd.to_datetime(df['License Expiration Date'])
df['License Creation Date'] = pd.to_datetime(df['License Creation Date'])

In [10]:
# Filter out inactive businesses
df = df.loc[df['License Status'] != 'Inactive']

In [11]:
# Keep only necessary columns
df = df[['Industry', 'Address ZIP', 'Longitude', 'Latitude', 'License Expiration Date', 'License Creation Date']]

In [12]:
df['Address ZIP'] = pd.to_numeric(df['Address ZIP'], errors='coerce')

In [13]:
# Remove businesses outside of New York City ZIP codes
df = df[(df['Address ZIP'] >= 10001) & (df['Address ZIP'] <= 11697)]

In [14]:
# Rename 'Address ZIP' to 'ZipCode' for consistency
df = df.rename(columns={'Address ZIP': 'ZipCode'})

In [15]:
# Group industries into broader categories
industry_categories = {
    'food_and_beverage': ['Catering Establishment', 'Sidewalk Cafe', 'Third Party Food Delivery'],
    'entertainment_and_recreation': ['Gaming Cafe', 'Amusement Arcade', 'Cabaret', 'Bingo Game Operator', 'Amusement Device Permanent', 'Amusement Device Portable', 'Amusement Device Temporary', 'Games of Chance', 'Sightseeing Bus', 'Sightseeing Guide', 'Pool or Billiard Room'],
    'parking_and_automotive_services': ['Car Wash', 'Booting Company', 'Garage', 'Garage and Parking Lot', 'Parking Lot', 'Secondhand Dealer - Auto', 'Tow Truck Company', 'Tow Truck Exemption', 'Tow Truck Driver'],
    'professional_services': ['Employment Agency', 'Electronic & Appliance Service', 'General Vendor Distributor', 'Process Serving Agency', 'Scale Dealer Repairer', 'Scrap Metal Processor', 'General Vendor', 'Process Server Individual'],
    'retail_services': ['Laundry', 'Laundries', 'Laundry Jobber', 'Newsstand', 'Electronics Store', 'Electronic Cigarette Dealer', 'Dealer In Products', 'Secondhand Dealer - Firearms', 'Secondhand Dealer - General', 'Tobacco Retail Dealer', 'Ticket Seller Business', 'Stoop Line Stand', 'Special Sale', 'Ticket Seller'],
    'financial_services': ['Debt Collection Agency', 'Pawnbroker'],
    'home_services': ['Construction Labor Provider', 'Home Improvement Contractor', 'Commercial Lessor', 'Auction House Premises', 'Storage Warehouse', 'Locksmith', 'Locksmith Apprentice', 'Home Improvement Salesperson', 'Auctioneer'],
    'transportation': ['Horse Drawn Cab Owner', 'Pedicab Business', 'Pedicab Driver', 'Horse Drawn Driver']
}

In [16]:
# Function to categorize industries
def categorize_industry(industry):
    for category, industries in industry_categories.items():
        if industry in industries:
            return category
    return 'other'

# Apply the categorization
df['BusinessType'] = df['Industry'].apply(categorize_industry)

In [17]:
# Aggregate data by zipcode and business type
business_by_zip = df.groupby(['ZipCode', 'BusinessType']).size().reset_index(name='Count')

In [18]:
# Sort the dataframe
business_by_zip = business_by_zip.sort_values(['ZipCode', 'BusinessType'])

In [19]:
business_by_zip

,ZipCode,BusinessType,Count
0,10001.0,entertainment_and_recreation,24
1,10001.0,financial_services,5
2,10001.0,home_services,97
3,10001.0,parking_and_automotive_services,41
4,10001.0,professional_services,46
...,...,...,...
1718,11694.0,retail_services,37
1719,11697.0,entertainment_and_recreation,1
1720,11697.0,home_services,8
1721,11697.0,professional_services,1


In [20]:
# Save to CSV
business_by_zip.to_csv('cleaned_business_data.csv', index=False)